In [104]:
import requests
import json
import boto3

client = boto3.client('sagemaker-runtime')

df = pd.read_csv("synthetic_data.csv")
# Convert DataFrame to a list of dictionaries
realtime_data = df.iloc[:2,:].to_dict(orient='records')

# Convert to JSON
json_data = json.dumps(realtime_data)

response = client.invoke_endpoint(
    EndpointName='prophet-endpoint',
    ContentType='application/json',
    Body=json.dumps(json_data)
)

predictions = response['Body'].read().decode()
# Clean the JSON string by removing extra escape characters
cleaned_response = predictions.replace('\\"', '"').strip('"\n')

In [105]:
cleaned_response

'[{"ds":1672531200000,"yhat":103.3200657555,"yhat_lower":100.7775545426,"yhat_upper":105.8792209069},{"ds":1672617600000,"yhat":103.219479347,"yhat_lower":100.5830584145,"yhat_upper":105.645204813}]'

In [106]:
# Convert to DataFrame
prediction_df = pd.DataFrame(json.loads(predictions))
prediction_df

ValueError: DataFrame constructor not properly called!

In [64]:
pip install pandas==1.5.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 107.2 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-expr 1.1.2 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [72]:
import joblib

data = pd.DataFrame(json.loads(json_data))

# Load the model
with open('prophet_model.pkl', 'rb') as f:
    model = pickle.load(f)

forecast = model.predict(data[['ds']])
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-01-01,103.386637,101.053947,105.886944,103.386637,103.386637,-0.066571,-0.066571,-0.066571,-0.066571,-0.066571,-0.066571,0.0,0.0,0.0,103.320066
